In [ ]:
"""
SHAP Interaction Values using 

From: Maya Thompson (City Mobility Manager)
To: Data Science Team
Subject: Understanding Bike-Share Demand Drivers

Hi Justin,

We’ve been looking at our bike-share ridership numbers and while we have some ideas about what drives

demand (like weather, time of day, and weekends), the team keeps running into confusing situations where things don’t add up.

On hot weekends, demand seems to spike — but not on hot weekdays.

Surge pricing seems to help sometimes, but not always.

Riders complain that even a small chance of rain kills demand, especially in humid weather.

Right now, our marketing and ops teams don’t know which combinations of factors matter most.

Could you build a quick analysis to show us not just the individual effects of these features,

but also how they interact with each other?

Ideally, we’d like some visuals we can share in our next planning meeting that

highlight the strongest pairs of features influencing ridership.

Thanks so much,
Maya

✍ **Author**: Justin Wall
📅 **Date**: 02/13/2025
"""

In [1]:
# ================================= #
# Generate Fake Bike Share Data     #
# ================================= #
#%%
# Regenerate with a small fix (use np.clip instead of ndarray.clip to avoid Index edge-cases)
import numpy as np
import pandas as pd
rng = np.random.default_rng(42)

n_days = 50
hours_per_day = 24
n = n_days * hours_per_day
start = pd.Timestamp("2024-06-01")

timestamps = pd.date_range(start, periods=n, freq="H")
hour_of_day = timestamps.hour
day_of_week = timestamps.dayofweek
is_weekend = ((day_of_week >= 5).astype(int))

daily_base = rng.normal(72, 8, size=n_days)
diurnal = 10 * np.sin(2 * np.pi * (hour_of_day - 15) / 24)
temp_F = np.repeat(daily_base, hours_per_day) + diurnal + rng.normal(0, 2.0, n)
temp_F = np.clip(temp_F, 35, 98)

humidity_base = rng.normal(60, 12, size=n) + 10 * np.cos(2 * np.pi * (hour_of_day) / 24)
humidity_base = np.clip(humidity_base, 20, 100)

rain_events = rng.binomial(1, 0.15, size=n_days)
rain_prob = np.repeat(rain_events, hours_per_day) * rng.beta(2, 5, size=n)
rain_prob = np.maximum(rain_prob, rng.binomial(1, 0.03, size=n) * rng.uniform(0.1, 0.6, size=n))

wind_mph = np.clip(rng.gamma(3, 2, size=n), 0, 30)

base_surge = (hour_of_day >= 17).astype(int) * rng.binomial(1, 0.4, size=n)
weekend_surge = is_weekend * rng.binomial(1, 0.30, size=n)
surge_pricing_on = np.clip(base_surge + weekend_surge, 0, 1)

holiday_days = rng.choice(np.arange(n_days), size=max(1, int(0.05 * n_days)), replace=False)
holiday = np.isin(np.repeat(np.arange(n_days), hours_per_day), holiday_days).astype(int)

near_park = np.ones(n, dtype=int)

humidity_pct = np.clip(humidity_base + 35 * (rain_prob > 0.4), 20, 100)

commute_wave = (
    18 * np.exp(-0.5 * ((hour_of_day - 8) / 2.5) ** 2) +
    28 * np.exp(-0.5 * ((hour_of_day - 18) / 3.0) ** 2)
)

beta_temp = 0.9 * np.maximum(temp_F - 50, 0)
beta_humidity = -0.08 * (humidity_pct - 60)
beta_wind = -0.7 * np.maximum(wind_mph - 8, 0)
beta_rain = -40 * rain_prob
beta_weekend = 10 * is_weekend
beta_surge = 6 * surge_pricing_on
beta_holiday = -8 * holiday

interaction_temp_weekend = 0.45 * (temp_F - 55) * is_weekend
interaction_surge_evening = 12 * surge_pricing_on * (hour_of_day >= 17)
interaction_rain_humid = -25 * rain_prob * np.clip((humidity_pct - 60) / 40, 0, 1)
interaction_wind_cold = -0.9 * wind_mph * (np.clip(60 - temp_F, 0, 20) / 20)

mu = (
    35 + commute_wave
    + beta_temp + beta_humidity + beta_wind + beta_rain + beta_weekend + beta_surge + beta_holiday
    + interaction_temp_weekend + interaction_surge_evening + interaction_rain_humid + interaction_wind_cold
)

epsilon = rng.normal(0, 6, size=n)
rides = np.clip(np.round(mu + epsilon), 0, None).astype(int)

df = pd.DataFrame({
    "timestamp": timestamps,
    "rides": rides,
    "temp_F": np.round(temp_F, 1),
    "humidity_pct": np.round(humidity_pct, 0).astype(int),
    "wind_mph": np.round(wind_mph, 1),
    "rain_prob": np.round(rain_prob, 3),
    "is_weekend": is_weekend,
    "hour_of_day": hour_of_day,
    "near_park": near_park,
    "surge_pricing_on": surge_pricing_on,
    "holiday": holiday
})

# path = "/mnt/data/bike_share_hourly.csv"
# df.to_csv(path, index=False)

# import caas_jupyter_tools as cj
# cj.display_dataframe_to_user("Bike-share synthetic dataset (hourly)", df.head(25))

# path

df.head()

#%%

C:\Users\wallj\AppData\Local\Temp\ipykernel_31268\989824901.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  timestamps = pd.date_range(start, periods=n, freq="H")


,timestamp,rides,temp_F,humidity_pct,wind_mph,rain_prob,is_weekend,hour_of_day,near_park,surge_pricing_on,holiday
0,2024-06-01 00:00:00,84,82.1,82,12.5,0.0,1,0,1,0,0
1,2024-06-01 01:00:00,95,80.7,88,8.1,0.0,1,1,1,0,0
2,2024-06-01 02:00:00,80,74.1,59,16.6,0.0,1,2,1,1,0
3,2024-06-01 03:00:00,60,73.8,71,4.0,0.0,1,3,1,0,0
4,2024-06-01 04:00:00,92,70.9,56,4.3,0.0,1,4,1,0,0


In [2]:
# =============================
# 1) SETUP & LOAD DATA
# =============================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import shap

# df = pd.read_csv("/mnt/data/bike_share_hourly.csv", parse_dates=["timestamp"])
print("Rows, Cols:", df.shape)
print(df.head())

Rows, Cols: (1200, 11)
            timestamp  rides  temp_F  humidity_pct  wind_mph  rain_prob  \
0 2024-06-01 00:00:00     84    82.1            82      12.5        0.0   
1 2024-06-01 01:00:00     95    80.7            88       8.1        0.0   
2 2024-06-01 02:00:00     80    74.1            59      16.6        0.0   
3 2024-06-01 03:00:00     60    73.8            71       4.0        0.0   
4 2024-06-01 04:00:00     92    70.9            56       4.3        0.0   

   is_weekend  hour_of_day  near_park  surge_pricing_on  holiday  
0           1            0          1                 0        0  
1           1            1          1                 0        0  
2           1            2          1                 1        0  
3           1            3          1                 0        0  
4           1            4          1                 0        0  


In [8]:
# =============================
# 2) FEATURE PREP & TRAIN/TEST SPLIT
# -----------------------------
# What: Select features, split into train and test.
# Why: Keep modeling honest and allow quick RMSE check.
# =============================
target = "rides"
features = ["temp_F",
            "humidity_pct",
            "wind_mph",
            "rain_prob",
            "is_weekend",
            "hour_of_day",
            "near_park",
            "surge_pricing_on",
            "holiday"]

X = df[features].copy()
y = df[target].values

# Train/test split (time-aware optional; here we do a random split for simplicity)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

print("Train:", X_train.shape, " Test:", X_test.shape)

Train: (960, 9)  Test: (240, 9)


In [9]:
# =============================
# 3) TRAIN A COMPACT TREE MODEL (XGBoost)
# -----------------------------
# What: Fit a small, readable XGBoost regressor.
# Why: Tree ensembles naturally capture interactions; SHAP has exact interaction values for trees.
# =============================
model = XGBRegressor(
    n_estimators=350,
    max_depth=4,
    learning_rate=0.08,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
    n_jobs=4
)
model.fit(X_train, y_train)

# Quick performance check
preds_test = model.predict(X_test)
rmse = mean_squared_error(y_test, preds_test)
print(f"Test RMSE: {rmse:.2f}")

Test RMSE: 50.75


In [5]:
print(model)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.08, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=350, n_jobs=4,
             num_parallel_tree=None, random_state=42, ...)


In [7]:
# =============================
# 4) SHAP EXPLAINER & INTERACTION VALUES
# -----------------------------
# What: Build a SHAP TreeExplainer, compute SHAP values and SHAP interaction values.
# Why: Interaction values reveal how PAIRS of features push predictions up/down together.
# =============================
# Compatibility patch for SHAP expecting deprecated numpy types
if not hasattr(np, "int"):
    np.int = int

# Reuse existing trained `model`, `X`
explainer = shap.TreeExplainer(model)

# # Subsample for speed
# rng = np.random.RandomState(0)
# sample_idx = rng.choice(len(X), size=min(800, len(X)), replace=False)
# X_shap = X.iloc[sample_idx]

# # Compute SHAP and interaction values
# shap_values = explainer.shap_values(X_shap)
# interaction_values = explainer.shap_interaction_values(X_shap)

# print("X_shap shape:", X_shap.shape)
# print("interaction_values shape:", np.array(interaction_values).shape)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 695: invalid start byte